# Setup

In [ ]:
# retrieve data
!wget 'https://s3.amazonaws.com/data-platform-tutorial/tweets.csv.gz'

In [ ]:
# uncompress data
!tar -xvzf tweets.csv.gz

In [ ]:
## standard python imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## pyspark imports
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

# create a spark session
spark = (SparkSession.builder
         .master("local")
         .appName("exploratory notebook")
         .enableHiveSupport()
         .getOrCreate())

# Loading Data

In [ ]:
# read csv as a dataframe
# df = (spark.read
#       .options(header=True, inferSchema=True)
#       .csv('tweets.csv'))

# Basic Exploration / Wrangling

### How many records do we have?

In [ ]:
## find the count of rows
# num_records = 

# print this number
print("{:,} records in the file".format(num_records))

### What are the columns and associated data types?

In [ ]:
## print the data types

### What does the data look like?

In [ ]:
## look at the first few rows
df.show(5, truncate=20)

### The `when` column is a unix timestamp with millisecond precision. How can we represent it as a timestamp?

In [ ]:
# create a new column 'ts', cast as a timestamp


# look at the data
df.show(5, truncate=25)

## Counts, Basic Visualization

### How many tweets were recorded for each term?

In [ ]:
# group by 'term' and count rows, show the result


### How can we visualize the above?

In [ ]:
# first output to pandas
# pandas_df = 

# set up figure, axes
fig, ax = plt.subplots(figsize=(10,6));

# use built-in plotting function from pandas
pandas_df.plot(x='term', y='count', kind='barh', legend=False, ax=ax);

# add axis labels
ax.set_xlabel("Total Count of Tweets", size=14);
ax.set_ylabel("Search Term", size=14);
ax.set_title("Count of Tweets by Search Term", size=20);

## Tweet Volume Over Time

### What does the volume of tweets look like over time?

In [ ]:
# group by 1 minute window, count
# counts_by_minute = 

# show the dataframe
counts_by_minute.show(10, truncate=100)

### Visualize the above data

In [ ]:
# first output to pandas

# use pandas to extract the first timestamp in the 'window' column
# and assign it to the 'time' column

# set up figure, axes
fig, ax = plt.subplots(figsize=(12,6))

# use built-in plotting function from pandas
pandas_df.plot(x='time', y='count', legend=None, ax=ax);

# add axis labels
ax.set_xlabel("Time", size=14);
ax.set_ylabel("# of Tweets", size=14);
ax.set_title("Tweets Per Minute", size=20);

## Tweet Volume Over Time, By Search Term

### What does the time series of each term look like?

In [ ]:
# group by term, and 1-minute window
# counts_by_minute_by_term = 

# show the dataframe
counts_by_minute_by_term.show(10, truncate=100)

### Visualize the above data

In [ ]:
# first output to pandas

# use pandas to extract the first timestamp in the 'window' column
# and assign it to the 'time' column
pandas_df['time'] = pandas_df['window'].apply(lambda w: w[0])

# set up figure, axes
fig, ax = plt.subplots(figsize=(12,6))

# plot each term individually
for term, group in pandas_df.groupby('term'):
    group.plot(x='time', y='count', label=term.upper(), ax=ax);

# set up axes labels
ax.set_xlabel("Time", size=14);
ax.set_ylabel("# of Tweets", size=14);
ax.set_title("Tweets Per Minute", size=20);

## Inspecting A Subset Of NBA Tweets

### What are people tweeting about at a particular time?

In [ ]:
# set the bounds of the time window we'd like to look at
start_time = '2017-05-23 03:45:00'
end_time = '2017-05-23 04:15:00'

# select nba tweets in this time window
# nba_tweets = 

# show the text of the first few tweets at that time
nba_tweets.select('value').show(10, truncate=100)

## What words are people using in these tweets?

### 1. Split tweets into arrays of words

In [ ]:
# use pyspark's built-in text processing tools
from pyspark.ml.feature import Tokenizer, StopWordsRemover

# split each tweet into it's constituent words
# tokenizer = 
# words_df = 

# show the result
words_df.select('words').show(10, truncate=100)

### 2. Count the words

In [ ]:
# count the resulting words
# exploded = 
# word_counts = 

# show the result, in descending order
word_counts.orderBy('count', ascending=False).show(10)

### 3. Get rid of common, non-informative words

In [ ]:
# use the StopWordsRemover tool to filter word arrays
# sw_remover = 
# filtered_words_df = 

# count the result
exploded = filtered_words_df.select(f.explode('filtered words').alias('word'))
filtered_word_counts = exploded.groupby('word').count()

# show the result, in descending order
filtered_word_counts.orderBy('count', ascending=False).show(10)